In [53]:
#| default_exp core

# core

> This package modifies some underlying fastHTML functionality so that Datastar is the underlying js library for DOM manipulation. This is currently implemented as a patch to the orignial library.
> FastHTML + Datastar Integration
> Server-driven DOM updates with clean Python APIs
> This module provides a seamless integration between FastHTML and Datastar, replacing HTMX's client-driven approach with Datastar's server-driven philosophy. Instead of defining swap behavior in HTML attributes, routes declare their patching behavior, and the server controls all DOM updates via Server-Sent Events.

In [54]:
#| hide
from nbdev.showdoc import *

In [55]:
#| export

from fastcore.utils import *
from fasthtml.core import _apply_ft
from fasthtml.common import *

from functools import partialmethod
from copy import deepcopy
from fasthtml.core import _wrap_call
import json


## Response Processing
*The core of the Datastar integration involves replacing FastHTML's response processing to generate Server-Sent Events (SSE) instead of traditional HTML responses.*

In [56]:
#| export
def _datastar_part_resp(req, resp):
    """Partition response into HTTP headers, background tasks, and content for Datastar compatibility"""
    resp = flat_tuple(resp)
    resp = resp + tuple(getattr(req, 'injects', ()))
    http_hdrs, resp = partition(resp, risinstance(HttpHeader))
    tasks, resp = partition(resp, risinstance(BackgroundTask))

    kw = {"headers": {}}  # Remove HTMX-specific vary headers
    if http_hdrs: kw['headers'] |= {o.k:str(o.v) for o in http_hdrs}
    if tasks:
        ts = BackgroundTasks()
        for t in tasks: ts.tasks.append(t)
        kw['background'] = ts

    resp = tuple(resp)
    if len(resp)==1: resp = resp[0]
    return resp, kw

This function processes FastHTML responses but removes HTMX-specific headers that aren't needed for Datastar:

In [57]:
from fasthtml.common import *
from starlette.testclient import TestClient


In [58]:
# Test with actual FastHTML app and TestClient
app = FastHTML()

@app.get("/test-headers")
def test_headers():
    return Div("Content"), HttpHeader("X-Custom", "test-value")

client = TestClient(app)
response = client.get("/test-headers")
assert response.headers["X-Custom"] == "test-value"

In [59]:
#| export  
def _datastar_xml(req, resp, indent):
    """Convert response to XML with intelligent header handling for full pages vs fragments"""
    is_datastar_request = (
        'datastar' in req.query_params or
        req.headers.get('datastar-request') == 'true'
    )

    if is_datastar_request:
        resp = _apply_ft(resp)
        return to_xml(resp, indent)
    else:
        hdr_tags = 'title','meta','link','style','base'
        resp = tuplify(resp)
        heads, bdy = partition(resp, lambda o: getattr(o, 'tag', '') in hdr_tags)
        title = [] if any(getattr(o, 'tag', '')=='title' for o in heads) else [Title(req.app.title)]
        canonical = [Link(rel="canonical", href=getattr(req, 'canonical', req.url))] if req.app.canonical else []
        resp = respond(req, [*heads, *title, *canonical], bdy)
        resp = _apply_ft(resp)
        return to_xml(resp, indent)

In [60]:
# Test fragment vs full page detection
app = FastHTML(title="Test App")

In [61]:
@app.get("/fragment")
def fragment():
    return Div(id="content")("Fragment content")

In [62]:
client = TestClient(app)

Regular request gets full page

In [63]:
response = client.get("/fragment")
assert "<title>Test App</title>" in response.text
assert "<div id=\"content\">Fragment content</div>" in response.text

Datastar request gets fragment only

In [64]:
response = client.get("/fragment?datastar=true")
# This would be just the fragment in a real implementation

In [65]:
#| export
def _datastar_resp(req, resp, cls=empty, status_code=200):
    """Create Datastar-compatible responses with automatic SSE formatting"""
    if not resp: resp = ''
    if hasattr(resp, '__response__'): resp = resp.__response__(req)
    if isinstance(resp, FileResponse) and not os.path.exists(resp.path):
        raise HTTPException(404, resp.path)

    resp, kw = _datastar_part_resp(req, resp)

    if cls is not empty: return cls(resp, status_code=status_code, **kw)
    if isinstance(resp, Response): return resp

    # Handle JSON responses as Datastar signals
    if isinstance(resp, Mapping):
        return JSONResponse(resp, status_code=status_code, **kw)

    # Check for Datastar patch options
    mode = getattr(req, 'datastar_mode', None)
    selector = getattr(req, 'datastar_selector', None)
    transition = getattr(req, 'datastar_transition', False)

    if mode or selector or transition:
        cts = _datastar_xml(req, resp, fh_cfg.indent)
        event_data = "event: datastar-patch-elements\n"
        if mode: event_data += f"data: mode {mode}\n"
        if selector: event_data += f"data: selector {selector}\n"
        if transition: event_data += f"data: useViewTransition true\n"
        event_data += f"data: elements {cts}\n\n"
        return Response(event_data, media_type="text/event-stream", status_code=status_code, **kw)
    else:
        cts = _datastar_xml(req, resp, fh_cfg.indent)
        return HTMLResponse(cts, status_code=status_code, **kw)

Test JSON responses become signals

In [66]:
app = FastHTML()

@app.get("/signals")
def signals():
    return {"counter": 42, "message": "hello"}

client = TestClient(app)
response = client.get("/signals")
assert response.json() == {"counter": 42, "message": "hello"}

## Route Enhancement

In [67]:
#| export
@patch
def route(self:FastHTML, path:str=None, methods=None, name=None, include_in_schema=True, body_wrap=None,
          patch_mode=None, selector=None, use_view_transition=False):
    """Enhanced route decorator with server-driven Datastar patch options"""
    def f(func):
        func._datastar_mode = patch_mode
        func._datastar_selector = selector  
        func._datastar_transition = use_view_transition
        return self._add_route(func, path, methods, name=name, include_in_schema=include_in_schema, body_wrap=body_wrap)
    return f(path) if callable(path) else f

Update HTTP method shortcuts

In [68]:
for method in all_meths:
    setattr(FastHTML, method, partialmethod(FastHTML.route, methods=method))

In [69]:
# Test route with patch options
app = FastHTML()

@app.get("/append-item", patch_mode="append", selector="#list")
def append_item():
    return Li("New item")

# Verify the function has Datastar attributes
assert hasattr(append_item, '_datastar_mode')
assert append_item._datastar_mode == "append"
assert append_item._datastar_selector == "#list"

## Endpoint Processing

In [70]:
#| export
@patch  
def _endp(self:FastHTML, f, body_wrap):
    """Enhanced endpoint wrapper with Datastar options support"""
    sig = signature_ex(f, True)
    async def _f(req):
        req.datastar_mode = getattr(f, '_datastar_mode', None)
        req.datastar_selector = getattr(f, '_datastar_selector', None)
        req.datastar_transition = getattr(f, '_datastar_transition', False)
        
        resp = None
        req.injects = []
        req.hdrs,req.ftrs,req.htmlkw,req.bodykw = map(deepcopy, (self.hdrs,self.ftrs,self.htmlkw,self.bodykw))
        req.hdrs,req.ftrs = listify(req.hdrs),listify(req.ftrs)
        
        for b in self.before:
            if not resp:
                if isinstance(b, Beforeware): bf,skip = b.f,b.skip
                else: bf,skip = b,[]
                if not any(re.fullmatch(r, req.url.path) for r in skip):
                    resp = await _wrap_call(bf, req, _params(bf))
        
        req.body_wrap = body_wrap
        if not resp: resp = await _wrap_call(f, req, sig.parameters)
        
        for a in self.after:
            _,*wreq = await _wrap_req(req, _params(a))
            nr = a(resp, *wreq)
            if nr: resp = nr
            
        return _datastar_resp(req, resp, sig.return_annotation)
    return _f

## Monkey Patch Application

In [74]:
#| export
import fasthtml.core
fasthtml.core._resp = _datastar_resp

In [75]:
#| hide
import nbdev; nbdev.nbdev_export()